In [1]:
from two4two.blender import Blender
from two4two.parameters import Parameters
import os

In [2]:
num_samples = 50000

In [3]:
object_variants = ['sticky', 'stretchy']
structure_variants = ['cubes', 'spheres', 'random']

In [4]:
save_dir = '/home/philipp/242/data'

In [5]:
n_processes = 16
chunk_size = 50

In [ ]:
for objects in object_variants:
    for structure in structure_variants:
        
        variant = objects + '_' + structure
        params_file = os.path.join(save_dir, variant+'.json')
        
        params = Parameters()
        params.generate_many(num_samples, params_file, objects, structure)
        
        output_dir = os.path.join(save_dir, variant)
        assert not os.path.isdir(output_dir)
        os.mkdir(output_dir)
        
        Blender(params_file, output_dir, n_processes, chunk_size)

Split /home/philipp/242/data/sticky_cubes.json into 1000 chunks.
Split /home/philipp/242/data/sticky_spheres.json into 1000 chunks.
